**Alunos: 
Jefferson Costa, 
Sávio Berdine, 
Amanda Lasserre,
Kevin Andrews,
Nicholas Henrique.**

In [ ]:
#-*- coding:utf-8 -*-
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
data = pd.read_csv("College.csv")
#No dataSet original, temos colunas com os nomes separados com um ".", por exemplo, Grad.Rate, e isso estava nos dando alguns problemas, então resolvemos renomear as colunas e trocar esse "." por um "_"
data = data.rename(columns={'Unnamed: 0':'College name','F.Undergrad':'F_Undergrad','P.Undergrad':'P_Undergrad','Room.Board':'Room_Board','S.F.Ratio':'S_F_Ratio','perc.alumni':'perc_alumni','Grad.Rate':'Grad_Rate'})
data.head()

In [ ]:
data.drop(["College name"], axis=1, inplace=True)
data.info()

**Os dados que serão utilizados**<br>
    **Private** A factor with levels No and Yes indicating private or public university     
    **Apps** Number of applications received     
    **Accept** Number of applications accepted     
    **Enroll Number** of new students enrolled     
    **Top10perc** Pct. new students from top 10% of H.S. class     
    **Top25perc** Pct. new students from top 25% of H.S. class     
    **F_Undergrad** Number of fulltime undergraduates     
    **P_Undergrad** Number of parttime undergraduates     
    **Outstate** Out-of-state tuition     
    **Room_Board** Room and board costs     
    **Books** Estimated book costs     
    **Personal** Estimated personal spending     
    **PhD Pct.** of faculty with Ph.D.’s     
    **Terminal Pct.** of faculty with terminal degree     
    **S_F_Ratio** Student/faculty ratio     
    **perc_alumni** Pct. alumni who donate     
    **Expend Instructional** expenditure per student     
    **Grad_Rate** Graduation rate<br>
**Observe que temos 18 variáveis**

### **Explorando o banco de dados**

**Uma comparação entre o número de estudantes "fulltime" e "parttime" de universidade privadas e universidades públicas.**

In [ ]:
plt.figure(figsize=(12, 8))

data.loc[data.Private == 'Yes', 'F_Undergrad'].hist(label="Universidades privadas F_Undergrad", bins=30)
data.loc[data.Private == 'Yes', 'P_Undergrad'].hist(label="Universidades privadas P_Undergrad", bins=30)
data.loc[data.Private == 'No', 'F_Undergrad'].hist(label="Universidades públicas F_Undergrad", bins=30)
data.loc[data.Private == 'No', 'P_Undergrad'].hist(label="Universidades privadas P_Undergrad", bins=30)

plt.xlabel('F_Undergrad vs P_Undergrad')
plt.legend()


In [ ]:
result = data['Private'].value_counts()
print(result)

**Uma comparação entre a taxa de graduação em universidades privadas e universidades públicas**

In [ ]:
plt.figure(figsize=(12, 8))

data.loc[data.Private == 'Yes', 'Grad_Rate'].hist(label="Universidades privadas", bins=30)
data.loc[data.Private == 'No', 'Grad_Rate'].hist(label="Universidades públicas", bins=30)

plt.xlabel('Taxa de graduação')
plt.legend()

Ao analisar o histograma, percebemos que tem uma universidade com uma taxa de graduação acima de 100%. Temos que saber qual é essa universidade e "setar" que a taxa de graduação é de 100%, pois isso faz mais sentido para nossa análise.


In [ ]:
data.loc[data.Grad_Rate > 100]

Ao fazer a busca, temos que a *Cazenovia College* é a universidade pela qual estamos procurando, então iremos 'setar' a taxa de graduação dela pra 100%.


In [ ]:
data.loc[data.Grad_Rate > 100, 'Grad_Rate'] = 100

**E agora faremos a comparação novamente**

In [ ]:
plt.figure(figsize=(12, 8))

data.loc[data.Private == 'Yes', 'Grad_Rate'].hist(label="Universidades privadas", bins=30)
data.loc[data.Private == 'No', 'Grad_Rate'].hist(label="Universidades públicas", bins=30)

plt.xlabel('Taxa de graduação')
plt.legend()

### Experimentos:

#### Para os experimentos serão utilizados os seguintes parâmetros da função "*KMeans*": *n_clusters, init, n_init, max_iter e random_state*.

*n_clusters*: Número de centróides que serão gerados. Padrão = 8.

*init*: Método para inicialização. Pode ser do tipo "*k_means++*", onde seleciona de maneira inteligente os centróides iniciais para acelerar a convergência, e do tipo "*random*", onde é escolhido aleatoriamente "*n_clusters*" linhas do banco de dados para serem os centróides iniciais. Padrão = "*k_means++*".

*n_init*: Número de vezes que o algoritmo K-Means executará com diferentes centróides iniciais. Padrão = 10.

*max_iter*: Número máximo de iterações que o algoritmo K-Means irá executar. Padrão = 300.

*random_state*: Número aleatório para geração inicial dos centróides. Padrão = *None*.

In [ ]:
#Transformando a columa em 1 pra yes e 0 pra no
data['Private'].replace(['Yes','No'],[1,0],inplace=True)

In [ ]:
#selecionando apenas as colunas com valores numéricos
X = data.iloc[:, [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]].values

**Criando uma instância do k-means com k = 2**

In [ ]:
kmeans = KMeans(n_clusters = 2)
#ajustando o k-means
y_kmeans = kmeans.fit_predict(X)

In [ ]:
print(classification_report(data.Private, kmeans.labels_))

Com esssa primeira instância do K-means, com o K = 2, não obtivemos um resultado muito bom. Faremos outros experimentos, para verificar o que acontece com o *precision*, *recall* e *accuracy*.

In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score
# data_investigation = pd.read_csv("College.csv")
# #No dataSet original, temos colunas com os nomes separados com um ".", por exemplo, Grad.Rate, e isso estava nos dando alguns problemas, então resolvemos renomear as colunas e trocar esse "." por um "_"
# data_investigation = data_investigation.rename(columns={'Unnamed: 0':'College name','F.Undergrad':'F_Undergrad','P.Undergrad':'P_Undergrad','Room.Board':'Room_Board','S.F.Ratio':'S_F_Ratio','perc.alumni':'perc_alumni','Grad.Rate':'Grad_Rate'})
# #Removendo a coluna que classifica as universidades
# data_investigation.drop('Private', axis='columns', inplace=True)
# X = data_investigation.iloc[:, [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]].values
for i, k in enumerate([2, 3, 4, 5, 6, 7]):
    fig, (ax1) = plt.subplots(1)
    fig.set_size_inches(18, 7)
    
    # Run the Kmeans algorithm
    km = KMeans(n_clusters=k)
    labels = km.fit_predict(X)
    centroids = km.cluster_centers_

    # Get silhouette samples
    silhouette_vals = silhouette_samples(X, labels)

    # Silhouette plot
    y_ticks = []
    y_lower, y_upper = 0, 0
    for i, cluster in enumerate(np.unique(labels)):
        cluster_silhouette_vals = silhouette_vals[labels == cluster]
        cluster_silhouette_vals.sort()
        y_upper += len(cluster_silhouette_vals)
        ax1.barh(range(y_lower, y_upper), cluster_silhouette_vals, edgecolor='none', height=1)
        ax1.text(-0.03, (y_lower + y_upper) / 2, str(i + 1))
        y_lower += len(cluster_silhouette_vals)

    # Get the average silhouette score and plot it
    avg_score = np.mean(silhouette_vals)
    ax1.axvline(avg_score, linestyle='--', linewidth=2, color='green')
    ax1.set_yticks([])
    ax1.set_xlim([-0.1, 1])
    ax1.set_xlabel('Silhouette coefficient values')
    ax1.set_ylabel('Cluster labels')
    ax1.set_title('Silhouette plot for the various clusters', y=1.02);
    
    plt.tight_layout
    plt.suptitle(f'Silhouette analysis using k = {k}',
                 fontsize=16, fontweight='semibold', y=1.05);

Consideramos que a quantidade de clusters ideal tem todos os clusters acima da média do silhouette score e um silhouette socore médio acima de 0.5. Fica claro que com dois clusters o silhouette socore médio tem o melhor resultado, ficando próximo a 0.6, entretanto um dos clusters tem o score sempre menor que a média, o que não é o ideal. Mais investigação deve ser feita sobre o data set.

#### Para os experimentos também serão utilizados os seguintes parâmetros da função "*train_test_split*": *train_size* e *random_state*.

*train_size*: Representa a porcentagem do banco de dados utilizada para treinamento, o restante fica alocado para teste. Padrão: *None*.

*random_state*: Número para aleatorizar as observações selecionadas do banco de dados *x* e *y*. Padrão: *None*.

In [ ]:
x = data.drop(['Private'],axis=1)
y = data["Private"]
y.value_counts()

#### Variando *train_size*:

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.50, random_state=None)
kmeans = KMeans(n_clusters=8, init="k-means++", n_init=10, max_iter=300, random_state = None)
kmeans.fit(x_train)
predicted = kmeans.predict(x_test)
print(accuracy_score(y_test, predicted))
print(classification_report(y_test,predicted))

Obtiveram-se esses *warnings* pois como foi passado *n_clusters=8*, o algoritmo "*KMeans*" tentará segregar o banco de dados que o foi passado em 8 diferentes classificações, entretanto o banco de dados utilizado possui apenas 2 classificações, "1" para faculadade privada e "0" para faculdade pública, logo, deve-se utilizar *n_clusters=2* para os experimentos.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.50, random_state=None)
kmeans = KMeans(n_clusters=2, init="k-means++", n_init=10, max_iter=300, random_state = None)
kmeans.fit(x_train)
predicted = kmeans.predict(x_test)
print(accuracy_score(y_test, predicted))
print(classification_report(y_test,predicted))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.6, random_state=None)
kmeans = KMeans(n_clusters=2, init="k-means++", n_init=10, max_iter=300, random_state = None)
kmeans.fit(x_train)
predicted = kmeans.predict(x_test)
print(accuracy_score(y_test, predicted))
print(classification_report(y_test,predicted))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.75, random_state=None)
kmeans = KMeans(n_clusters=2, init="k-means++", n_init=10, max_iter=300, random_state = None)
kmeans.fit(x_train)
predicted = kmeans.predict(x_test)
print(accuracy_score(y_test, predicted))
print(classification_report(y_test,predicted))

*train_size=0.75* apresentou o melhor resultado.

#### Variando *random_state* de *train_test_split*:

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.75, random_state=0)
kmeans = KMeans(n_clusters=2, init="k-means++", n_init=10, max_iter=300, random_state = None)
kmeans.fit(x_train)
predicted = kmeans.predict(x_test)
print(accuracy_score(y_test, predicted))
print(classification_report(y_test,predicted))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.75, random_state=1)
kmeans = KMeans(n_clusters=2, init="k-means++", n_init=10, max_iter=300, random_state = None)
kmeans.fit(x_train)
predicted = kmeans.predict(x_test)
print(accuracy_score(y_test, predicted))
print(classification_report(y_test,predicted))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.75, random_state=15)
kmeans = KMeans(n_clusters=2, init="k-means++", n_init=10, max_iter=300, random_state = None)
kmeans.fit(x_train)
predicted = kmeans.predict(x_test)
print(accuracy_score(y_test, predicted))
print(classification_report(y_test,predicted))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.75, random_state=42)
kmeans = KMeans(n_clusters=2, init="k-means++", n_init=10, max_iter=300, random_state = None)
kmeans.fit(x_train)
predicted = kmeans.predict(x_test)
print(accuracy_score(y_test, predicted))
print(classification_report(y_test,predicted))

O *random_state=0* apresentou os melhores resultados.

#### Variando *init*:

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.75, random_state=0)
kmeans = KMeans(n_clusters=2, init="random", n_init=10, max_iter=300, random_state = None)
kmeans.fit(x_train)
predicted = kmeans.predict(x_test)
print(accuracy_score(y_test, predicted))
print(classification_report(y_test,predicted))

Obtiveram-se os mesmos resultados.

#### Variando *n_init*:

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.75, random_state=0)
kmeans = KMeans(n_clusters=2, init="k-means++", n_init=5, max_iter=300, random_state = None)
kmeans.fit(x_train)
predicted = kmeans.predict(x_test)
print(accuracy_score(y_test, predicted))
print(classification_report(y_test,predicted))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.75, random_state=0)
kmeans = KMeans(n_clusters=2, init="k-means++", n_init=15, max_iter=300, random_state = None)
kmeans.fit(x_train)
predicted = kmeans.predict(x_test)
print(accuracy_score(y_test, predicted))
print(classification_report(y_test,predicted))

A alteração de *n_init* não proporcionou uma melhora no resultado.

#### Variando *max_iter*:

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.75, random_state=0)
kmeans = KMeans(n_clusters=2, init="k-means++", n_init=10, max_iter=50, random_state = None)
kmeans.fit(x_train)
predicted = kmeans.predict(x_test)
print(accuracy_score(y_test, predicted))
print(classification_report(y_test,predicted))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.75, random_state=0)
kmeans = KMeans(n_clusters=2, init="k-means++", n_init=10, max_iter=100, random_state = None)
kmeans.fit(x_train)
predicted = kmeans.predict(x_test)
print(accuracy_score(y_test, predicted))
print(classification_report(y_test,predicted))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.75, random_state=0)
kmeans = KMeans(n_clusters=2, init="k-means++", n_init=10, max_iter=400, random_state = None)
kmeans.fit(x_train)
predicted = kmeans.predict(x_test)
print(accuracy_score(y_test, predicted))
print(classification_report(y_test,predicted))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.75, random_state=0)
kmeans = KMeans(n_clusters=2, init="k-means++", n_init=10, max_iter=500, random_state = None)
kmeans.fit(x_train)
predicted = kmeans.predict(x_test)
print(accuracy_score(y_test, predicted))
print(classification_report(y_test,predicted))

Um aumento no valor de *max_iter* resultou em uma piora dos resultados.

#### Variando *random_state* da função "*KMeans*":

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.75, random_state=0)
kmeans = KMeans(n_clusters=2, init="k-means++", n_init=10, max_iter=300, random_state = 0)
kmeans.fit(x_train)
predicted = kmeans.predict(x_test)
print(accuracy_score(y_test, predicted))
print(classification_report(y_test,predicted))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.75, random_state=0)
kmeans = KMeans(n_clusters=2, init="k-means++", n_init=10, max_iter=300, random_state = 1)
kmeans.fit(x_train)
predicted = kmeans.predict(x_test)
print(accuracy_score(y_test, predicted))
print(classification_report(y_test,predicted))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.75, random_state=0)
kmeans = KMeans(n_clusters=2, init="k-means++", n_init=10, max_iter=300, random_state = 15)
kmeans.fit(x_train)
predicted = kmeans.predict(x_test)
print(accuracy_score(y_test, predicted))
print(classification_report(y_test,predicted))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.75, random_state=0)
kmeans = KMeans(n_clusters=2, init="k-means++", n_init=10, max_iter=300, random_state = 42)
kmeans.fit(x_train)
predicted = kmeans.predict(x_test)
print(accuracy_score(y_test, predicted))
print(classification_report(y_test,predicted))

Os resultados de *random_state=None* apresentaram-se os melhores entre os testados. O valor *None* significa que será usado a instância de estado aleatório global de *numpy.random*.

Portanto, os melhores resultados foram obtidos quando foram utilizados os parâmetros: *train_size=0.75*, *random_state* de *train_test_split* igual a 0 e *n_clusters=2*, *init=k-means++*, *n_init=10*, *max_iter=300* e *random_state* de *KMeans* igual a *None*.

### Tenta-se agora descobrir quais são as melhores variáveis para utilizar no banco de dados para tentar maximizar os resultados.

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model = ExtraTreesClassifier()
model.fit(x,y)
feat_importances = pd.Series(model.feature_importances_, index=x.columns)
feat_importances.nlargest(12).plot(kind='barh')
plt.show()

### Irão ser utilizados somente as colunas "*F.Undergrad*", "*Outstate*" e "*Enroll*".

In [ ]:
data2 = pd.read_csv("College.csv")
data2.drop(["Unnamed: 0", "Top10perc", "Expend", "Grad.Rate", "P.Undergrad", "Room.Board", "perc.alumni", "Accept", "Apps", "S.F.Ratio"], axis=1, inplace=True)
data2["Private"].replace(["Yes", "No"], [1,0], inplace=True)
x2 = data.drop(['Private'],axis=1)
y2 = data["Private"]

### Realiza-se o experimento com os melhores parâmetros que foram obtidos anteriormente:

In [ ]:
x2_train, x2_test, y2_train, y2_test = train_test_split(x2, y2, train_size=0.75, random_state=0)
kmeans2 = KMeans(n_clusters=2, init="k-means++", n_init=10, max_iter=300, random_state = None)
kmeans2.fit(x2_train)
predicted2 = kmeans.predict(x2_test)
print(accuracy_score(y2_test, predicted2))
print(classification_report(y2_test,predicted2))

Infelizmente com esse novo banco de dados não foi possível melhorar os resultados.